<a href="https://colab.research.google.com/github/Jimmy586/MachineLearningJourney/blob/master/DBScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install opendatasets

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import opendatasets as od
import pandas as pd
import numpy as np
import random

In [ ]:
### Data Preprocesiong

od.download("https://www.kaggle.com/datasets/whenamancodes/predict-diabities")

df=pd.read_csv('/content/predict-diabities/diabetes.csv')

df

print(df.isnull().sum())

y = df['Outcome']
feature = df.iloc[:,0:8]
feature = np.array(feature)

norm = MinMaxScaler()
norm_feature = norm.fit_transform(feature)
y = np.array(y)




Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:


Abort: ignored

In [ ]:

## DB_Scan Implementation

### using Numpy + eps on euclidean distance

def check_core_point(eps,minPts, data, index):
    core=False
    border=False
    Noise=False
    data_point = data[index]
    indexes = np.arange( len(data) )
    dist = np.linalg.norm(data - data_point,axis=1)
    neighbours = (dist <= eps)
    neighbours[index] = False
    neighbours_indexes = indexes[neighbours]


    if len(neighbours_indexes) >= minPts:
        core=True
    elif (len(neighbours_indexes) < minPts) and len(neighbours_indexes) > 0:
        border=True
    elif len(neighbours_indexes) == 0:
        Noise=True
    return (neighbours_indexes , core, border, Noise)

### using Numpy + eps on every feature

def check_core_point(eps,minPts, data, index):
    core=False
    border=False
    Noise=False
    data_point = data[index]
    indexes = np.arange( len(data) )
    dist = abs(data - data_point)
    neighbours = np.all((dist <= eps), axis = 1)
    neighbours[index] = False
    neighbours_indexes = indexes[neighbours]


    if len(neighbours_indexes) >= minPts:
        core=True
    elif (len(neighbours_indexes) < minPts) and len(neighbours_indexes) > 0:
        border=True
    elif len(neighbours_indexes) == 0:
        Noise=True
    return (neighbours_indexes , core, border, Noise)

### using dataframe + eps on every feature

def check_core_point(eps,minPts, df, index):
    core=False
    border=False
    Noise=False
    x = df.iloc[index][df.columns[0]]

    temp = (np.abs(x - df[ df.columns[0] ]) <= eps) & (df.index != index)

    for c in df.columns[1:]:
      x = df.iloc[index][c]
      temp =  ((np.abs(x - df[c]) <= eps) & temp)

    temp = df[temp]

    if len(temp) >= minPts:
        core=True
    elif (len(temp) < minPts) and len(temp) > 0:
        border=True
    elif len(temp) == 0:
        Noise=True
    return (temp , core, border, Noise)

### main rules Numpy

def cluster_with_stack(eps, minPts, df):

    C = 1
    current_stack = set()
    unvisited = list(np.arange(len(df)))
    clusters = []


    while (len(unvisited) != 0):

        first_point = True

        current_stack.add(random.choice(unvisited))

        while len(current_stack) != 0:

            curr_idx = current_stack.pop()

            neigh_indexes, iscore, isborder, isnoise = check_core_point(eps, minPts, df, curr_idx)

            if (isborder & first_point):
                clusters.append((curr_idx, 0))
                clusters.extend(list(zip(neigh_indexes,[0 for _ in range(len(neigh_indexes))])))

                unvisited.remove(curr_idx)
                unvisited = [e for e in unvisited if e not in neigh_indexes]

                continue

            unvisited.remove(curr_idx)


            neigh_indexes = set(neigh_indexes) & set(unvisited)

            if iscore:
                first_point = False

                clusters.append((curr_idx,C))
                current_stack.update(neigh_indexes)

            elif isborder:
                clusters.append((curr_idx,C))

                continue

            elif isnoise:
                clusters.append((curr_idx, 0))

                continue

        if not first_point:
            C+=1

    return clusters

### main rules dataframe

def cluster_with_stack(eps, minPts, df):

    C = 1
    current_stack = set()
    unvisited = list(df.index)
    clusters = []


    while (len(unvisited) != 0):


        first_point = True

        current_stack.add(random.choice(unvisited))

        while len(current_stack) != 0:


            curr_idx = current_stack.pop()


            neigh_indexes, iscore, isborder, isnoise = check_core_point(eps, minPts, df, curr_idx)

            if (isborder & first_point):
                clusters.append((curr_idx, 0))
                clusters.extend(list(zip(neigh_indexes,[0 for _ in range(len(neigh_indexes))])))

                unvisited.remove(curr_idx)
                unvisited = [e for e in unvisited if e not in neigh_indexes]

                continue

            unvisited.remove(curr_idx)


            neigh_indexes = set(neigh_indexes) & set(unvisited)

            if iscore:
                first_point = False

                clusters.append((curr_idx,C))
                current_stack.update(neigh_indexes)

            elif isborder:
                clusters.append((curr_idx,C))

                continue

            elif isnoise:
                clusters.append((curr_idx, 0))

                continue

        if not first_point:
            C+=1

    return clusters

### Accuracy Function

def acc(x,y):
  noise_idx = np.array([t[0] for t in x.values if t[1] == 0])
  c1_pred = np.array([t[0] for t in x.values if t[1] == 1])
  c1_pred = np.concatenate((c1_pred,noise_idx),axis=0)
  c2_pred = np.array([t[0] for t in x.values if t[1] == 2])

  c1 = np.array([t for t,i in enumerate(y) if i == 0])
  c2 = np.array([t for t,i in enumerate(y) if i == 1])

  res = (len( np.intersect1d(c1 , c1_pred) ) + len(np.intersect1d(c2 , c2_pred)) ) /len(y) *100
  if(res < 50): return 100 - res
  return res








In [ ]:
### without PCA

eps = 0.3
minPts = 8

clustered = cluster_with_stack(eps, minPts, norm_feature)

idx , cluster = list(zip(*clustered))

np.unique(cluster)

cluster_df = pd.DataFrame(clustered, columns = ["idx", "cluster"])
acc(cluster_df,y)

In [ ]:

## testing the code



### with PCA

cov_matrix = np.cov(feature.T)
eigen_values, eigen_vectors = np.linalg.eig(cov_matrix)
sorted_index = np.argsort(eigen_values)[::-1]
print (sorted_index)

sorted_eigenvalue = eigen_values[sorted_index]
sorted_eigenvectors = eigen_vectors[:,sorted_index]

n_components = 2
eigenvector_subset = sorted_eigenvectors[:,0:n_components]

X_reduced = np.dot(eigenvector_subset.transpose(),feature.transpose()).transpose()

norm_pca = norm.fit_transform(X_reduced)

eps = 0.2
minPts = 3

clustered = cluster_with_stack(eps, minPts, norm_pca)
idx , cluster = list(zip(*clustered))
np.unique(cluster)

cluster_df = pd.DataFrame(clustered, columns = ["idx", "cluster"])

acc(cluster_df,y)

